In [6]:
import pandas as pd
from functions import assembleDf, epochG, epochsG
from datetime import datetime, timedelta
import numpy as np
import sys
sys.path.append('..')
from pyglicko2.glicko2_tests import exampleCase
from pyglicko2.glicko2 import Player
import glicko2
import time


In [4]:
matches = pd.read_csv('../Data/matches_glicko2.csv',parse_dates = ['tourney_date'], 
                      infer_datetime_format = True)

In [7]:
epochG(matches[1:10],{},matches.iloc[10]['tourney_date'])

({113953: <pyglicko2.glicko2.Player at 0x7fe3b0e2e820>,
  114146: <pyglicko2.glicko2.Player at 0x7fe3b0e2e790>,
  113987: <pyglicko2.glicko2.Player at 0x7fe381cdc8b0>,
  114147: <pyglicko2.glicko2.Player at 0x7fe381cdc490>,
  114149: <pyglicko2.glicko2.Player at 0x7fe381cdc520>,
  114154: <pyglicko2.glicko2.Player at 0x7fe381cdc2b0>,
  113963: <pyglicko2.glicko2.Player at 0x7fe381cdcbb0>,
  114158: <pyglicko2.glicko2.Player at 0x7fe381cdc2e0>,
  113999: <pyglicko2.glicko2.Player at 0x7fe381cdcf70>,
  113976: <pyglicko2.glicko2.Player at 0x7fe381cdcb80>,
  113982: <pyglicko2.glicko2.Player at 0x7fe381cdc130>},
 {(113987, Timestamp('1877-07-09 00:00:00')): (1799.6258356900184,
   227.73541461924287),
  (113963, Timestamp('1877-07-09 00:00:00')): (1599.8752635134226,
   227.73539746084663),
  (113999, Timestamp('1877-07-09 00:00:00')): (1599.8752635134226,
   227.73539746084663),
  (114158, Timestamp('1877-07-09 00:00:00')): (1500.0, 253.4045963894164),
  (114149, Timestamp('1877-07-09 00

In [ ]:
epochsG(matches[250_000:400_000],365)

note: it's not yet clear how to compute win probabilities from the glicko2 rating system.

In [ ]:
range(0,max(matches.index),100)

In [78]:
%%time
start = time.time()
pd,rh = epochsG(matches[0:100_000],180)
compute_time = time.time()-start
print(compute_time)


9.954494953155518
CPU times: user 9.92 s, sys: 41.5 ms, total: 9.96 s
Wall time: 9.95 s


In [1]:
rh

NameError: name 'rh' is not defined